In [1]:
# Import necessary libraries
import numpy as np
import torch
from torch import nn
from torch.nn import Transformer
from torch.utils.data import DataLoader, TensorDataset
import pickle

## NanoGPS adds
from torch.nn import functional as F

In [2]:
print("CUDA Available: ",torch.cuda.is_available())
use_cuda=True
device = torch.device("cuda:0" if use_cuda and torch.cuda.is_available() else "cpu")
print("device", device)

CUDA Available:  True
device cuda:0


In [3]:
# "AB","BC","CD","DE","EF","FG"# LS 
# "BA","CA","DA","EA","FA","GA"# MTO
# "AB","AC","AD","AE","AF","AG"# OTM

In [4]:
%run ../utils/create_trials.py "AB","AC","AD","AE","AF","AG"

Trainig pairs for the experiment ['AB', 'AC', 'AD', 'AE', 'AF', 'AG']
Creating experiment with 7 members, 4 classes and 21 dummy stimulus.
Experiment trials created!


In [5]:
trials_info_columns=["st_sample","st_comp1","st_comp2","st_comp3","option_answer"]#"st_comparison"

train_trials_corpus_df=train_info[trials_info_columns]
train_dummy_trials_corpus_df=train_dummy_info[trials_info_columns]#"st_comparison"

In [6]:
train_info.sample(20)

,sample_subset,st_sample,st_comp1,st_comp2,st_comp3,option_answer,st_comparison
4655,train,A4,A2,G1,B4,O_3,B4
2331,train,A2,B2,F1,A1,O_1,B2
16563,train,A2,E2,D4,E4,O_1,E2
16050,train,A1,E1,A4,E3,O_1,E1
14348,train,A4,E3,G2,D4,O_3,D4
1830,train,A2,B2,G1,F4,O_1,B2
16810,train,A2,E3,E2,D4,O_2,E2
27079,train,A2,F3,G2,A3,O_2,G2
22966,train,A3,G2,F3,F1,O_2,F3
20971,train,A1,F3,F1,E3,O_2,F1


In [7]:
# tokens_list=list(stims_random.keys())
tokens_list=sorted(list(set((list(np.array(train_trials_corpus_df).reshape([1,-1])[0])+(list(np.array(train_dummy_trials_corpus_df).reshape([1,-1])[0]))))))

In [8]:
token_to_index = {token: index for index, token in enumerate(tokens_list)}

In [9]:
# encode corpus in positional tokens train data Dummy comparison
encoded_training_data = np.array([[token_to_index[token] for token in sequence] for sequence in np.array(train_trials_corpus_df)])
train_tensor_encoded=torch.from_numpy(encoded_training_data).int().to(device)
train_tensor_encoded=train_tensor_encoded.to(torch.long)

In [10]:
# encode corpus in positional tokens Train data class member comparison
encoded_training_data_dummy = np.array([[token_to_index[token] for token in sequence] for sequence in np.array(train_dummy_trials_corpus_df)])
train_dummy_tensor_encoded=torch.from_numpy(encoded_training_data_dummy).int().to(device)
train_dummy_tensor_encoded=train_dummy_tensor_encoded.to(torch.long)

In [11]:
# set seed for reproductibility
torch.manual_seed(183)

# hyperparameters
batch_size = 64 # how many independent sequences will we process in parallel?
block_size = 4 # what is the maximum context length for predictions?
max_iters = 5000
eval_interval = 500
learning_rate = 3e-4
eval_iters = 200

n_embd = 384
n_head = 6
n_layer = 6
dropout = 0.2

In [12]:
# here are all the unique characters that occur in this text
chars = tokens_list
vocab_size = len(chars)
# create a mapping from characters to integers
stoi = { ch:i for i,ch in enumerate(chars) }
itos = { i:ch for i,ch in enumerate(chars) }
encode = lambda s: [stoi[c] for c in s] # encoder: take a string, output a list of integers
decode = lambda l: [itos[i] for i in l]#lambda l: ','.join([itos[i] for i in l]) # decoder: take a list of integers, output a string

In [13]:
# data loading
def get_batch(data):
    # data must be train trials encoded and transformed in torch tensor
    # generate a small batch of data of inputs x and targets y
    # data = train_tensor_encoded #if split == 'train' else val_data
    ix = torch.randint(len(data), (batch_size,)) 
    x = torch.stack([data[i,:block_size] for i in ix])
    y = torch.stack([data[i,1:] for i in ix])
    x, y = x.to(device), y.to(device)
    return x, y

@torch.no_grad()
def estimate_loss():
    out = {}
    model.eval()
    # data = train_tensor_encoded# Modify for train several models
    losses = torch.zeros(eval_iters)
    for k in range(eval_iters):
        X, Y = get_batch(data)
        logits, loss = model(X, Y)
        losses[k] = loss.item()
    out["train"] = losses.mean()
    model.train()
    return out

In [14]:
class Head(nn.Module):
    """ one head of self-attention """

    def __init__(self, head_size):
        super().__init__()
        self.key = nn.Linear(n_embd, head_size, bias=False)
        self.query = nn.Linear(n_embd, head_size, bias=False)
        self.value = nn.Linear(n_embd, head_size, bias=False)
        self.register_buffer('tril', torch.tril(torch.ones(block_size, block_size)))

        self.dropout = nn.Dropout(dropout)

    def forward(self, x):
        # input of size (batch, time-step, channels)
        # output of size (batch, time-step, head size)
        B,T,C = x.shape
        k = self.key(x)   # (B,T,hs)
        q = self.query(x) # (B,T,hs)
        # compute attention scores ("affinities")
        wei = q @ k.transpose(-2,-1) * k.shape[-1]**-0.5 # (B, T, hs) @ (B, hs, T) -> (B, T, T)
        # wei = wei.masked_fill(self.tril[:T, :T] == 0, float('-inf')) # (B, T, T) #filter communication with past# comment this and you have an encoder block
        wei = F.softmax(wei, dim=-1) # (B, T, T)
        wei = self.dropout(wei)
        # perform the weighted aggregation of the values
        v = self.value(x) # (B,T,hs)
        out = wei @ v # (B, T, T) @ (B, T, hs) -> (B, T, hs)
        return out

class MultiHeadAttention(nn.Module):
    """ multiple heads of self-attention in parallel """

    def __init__(self, num_heads, head_size):
        super().__init__()
        self.heads = nn.ModuleList([Head(head_size) for _ in range(num_heads)])
        self.proj = nn.Linear(head_size * num_heads, n_embd)
        self.dropout = nn.Dropout(dropout)

    def forward(self, x):
        out = torch.cat([h(x) for h in self.heads], dim=-1)
        out = self.dropout(self.proj(out))
        return out

class FeedFoward(nn.Module):
    """ a simple linear layer followed by a non-linearity """

    def __init__(self, n_embd):
        super().__init__()
        self.net = nn.Sequential(
            nn.Linear(n_embd, 4 * n_embd),
            nn.ReLU(),
            nn.Linear(4 * n_embd, n_embd),
            nn.Dropout(dropout),
        )

    def forward(self, x):
        return self.net(x)

class Block(nn.Module):
    """ Transformer block: communication followed by computation """

    def __init__(self, n_embd, n_head):
        # n_embd: embedding dimension, n_head: the number of heads we'd like
        super().__init__()
        head_size = n_embd // n_head
        self.sa = MultiHeadAttention(n_head, head_size)
        self.ffwd = FeedFoward(n_embd)
        self.ln1 = nn.LayerNorm(n_embd)
        self.ln2 = nn.LayerNorm(n_embd)

    def forward(self, x):
        x = x + self.sa(self.ln1(x))
        x = x + self.ffwd(self.ln2(x))
        return x

class GPTLanguageModel(nn.Module):

    def __init__(self):
        super().__init__()
        # each token directly reads off the logits for the next token from a lookup table
        self.token_embedding_table = nn.Embedding(vocab_size, n_embd)
        self.position_embedding_table = nn.Embedding(block_size, n_embd)
        self.blocks = nn.Sequential(*[Block(n_embd, n_head=n_head) for _ in range(n_layer)])
        self.ln_f = nn.LayerNorm(n_embd) # final layer norm
        self.lm_head = nn.Linear(n_embd, vocab_size)

        # better init, not covered in the original GPT video, but important, will cover in followup video
        self.apply(self._init_weights)

    def _init_weights(self, module):
        if isinstance(module, nn.Linear):
            torch.nn.init.normal_(module.weight, mean=0.0, std=0.02)
            if module.bias is not None:
                torch.nn.init.zeros_(module.bias)
        elif isinstance(module, nn.Embedding):
            torch.nn.init.normal_(module.weight, mean=0.0, std=0.02)

    def forward(self, idx, targets=None):
        B, T = idx.shape

        # idx and targets are both (B,T) tensor of integers
        tok_emb = self.token_embedding_table(idx) # (B,T,C)
        pos_emb = self.position_embedding_table(torch.arange(T, device=device)) # (T,C)
        x = tok_emb + pos_emb # (B,T,C)
        x = self.blocks(x) # (B,T,C)
        x = self.ln_f(x) # (B,T,C)
        logits = self.lm_head(x) # (B,T,vocab_size)

        if targets is None:
            loss = None
        else:
            B, T, C = logits.shape
            logits = logits.view(B*T, C)
            targets = targets.view(B*T)
            loss = F.cross_entropy(logits, targets)

        return logits, loss

    def generate(self, idx, max_new_tokens):
        # idx is (B, T) array of indices in the current context
        for _ in range(max_new_tokens):
            # crop idx to the last block_size tokens
            idx_cond = idx[:, -block_size:]
            # get the predictions
            logits, loss = self(idx_cond)
            # focus only on the last time step
            logits = logits[:, -1, :] # becomes (B, C)
            # apply softmax to get probabilities
            probs = F.softmax(logits, dim=-1) # (B, C)
            # sample from the distribution
            idx_next = torch.multinomial(probs, num_samples=1) # (B, 1)
            # append sampled index to the running sequence
            idx = torch.cat((idx, idx_next), dim=1) # (B, T+1)
        return idx

In [15]:
def get_trials_answers(tensor_encoded, model_trained, tell_me=True, tell_me_n_trials=1000):
    # tensor_encoded: Pytorch tensor long(float64) type [sample, comp_1, comp_2, comp_3, comp_response] of all group trials
    max_iters=tensor_encoded.shape[0]
    response_stimulus=[]
    for count, trial in enumerate(tensor_encoded[:,:-1]):
        tokens_response=model_trained.generate(trial.reshape([1,-1]), max_new_tokens=1)[0].tolist()
        response=decode(tokens_response)[-1]
        response_stimulus.append(response)

        if tell_me:
            # tell me the progress every tell_me_n_trials
            if count % tell_me_n_trials == 0 or count == max_iters - 1:
                print(f"{count} ({100.0*(count/max_iters):.2f}%) trials processed")
    return response_stimulus

In [16]:
########### Model_1 train trials class comparison

In [17]:
model_class = GPTLanguageModel()
m_class = model_class.to(device)
# print the number of parameters in the model
print(sum(p.numel() for p in m_class.parameters())/1e6, 'M parameters')

data=train_tensor_encoded
model=m_class

# create a PyTorch optimizer
optimizer = torch.optim.AdamW(model_class.parameters(), lr=learning_rate)

for iter in range(max_iters):

    # every once in a while evaluate the loss on train and val sets
    if iter % eval_interval == 0 or iter == max_iters - 1:
        losses = estimate_loss()
        # print(losses)
        print(f"step {iter}: train loss {losses['train']:.4f}")

    # sample a batch of data
    xb, yb = get_batch(data)

    # evaluate the loss
    logits, loss = model_class(xb, yb)
    optimizer.zero_grad(set_to_none=True)
    loss.backward()
    optimizer.step()

10.682164 M parameters
step 0: train loss 4.0410
step 500: train loss 0.0011
step 1000: train loss 0.0004
step 1500: train loss 0.0001
step 2000: train loss 0.0001
step 2500: train loss 0.0001
step 3000: train loss 0.0002
step 3500: train loss 0.0002
step 4000: train loss 0.0000
step 4500: train loss 0.0000
step 4999: train loss 0.0000


In [18]:
#### Predict model data

# train_responses=get_trials_answers(train_tensor_encoded, m_class)
# train_responses_info=train_info.copy()
# train_responses_info["response"]=train_responses
# train_responses_info["response_val"]=(train_responses_info.st_comparison==train_responses_info.response)*1

# sum(train_responses_info.response_val)/train_responses_info.shape[0]

In [19]:
########### Model_2 train trials dummy comparison

In [20]:
model_dummy = GPTLanguageModel()
m_dummy = model_dummy.to(device)
# print the number of parameters in the model
print(sum(p.numel() for p in m_dummy.parameters())/1e6, 'M parameters')

data=train_dummy_tensor_encoded
model=m_dummy

# create a PyTorch optimizer
optimizer = torch.optim.AdamW(model_dummy.parameters(), lr=learning_rate)

for iter in range(max_iters):

    # every once in a while evaluate the loss on train and val sets
    if iter % eval_interval == 0 or iter == max_iters - 1:
        losses = estimate_loss()
        # print(losses)
        print(f"step {iter}: train loss {losses['train']:.4f}")

    # sample a batch of data
    xb, yb = get_batch(data)

    # evaluate the loss
    logits, loss = model_dummy(xb, yb)
    optimizer.zero_grad(set_to_none=True)
    loss.backward()
    optimizer.step()

10.682164 M parameters
step 0: train loss 4.0001
step 500: train loss 0.0012
step 1000: train loss 0.0004
step 1500: train loss 0.0002
step 2000: train loss 0.0001
step 2500: train loss 0.0001
step 3000: train loss 0.0001
step 3500: train loss 0.0000
step 4000: train loss 0.0002
step 4500: train loss 0.0000
step 4999: train loss 0.0000


In [21]:
# ### Predict model data

# train_dummy_responses=get_trials_answers(train_dummy_tensor_encoded, m_dummy)

# train_dummy_responses_info=train_dummy_info.copy()
# train_dummy_responses_info["response"]=train_dummy_responses
# train_dummy_responses_info["response_val"]=(train_dummy_responses_info.st_comparison==train_dummy_responses_info.response)*1

# sum(train_dummy_responses_info.response_val)/train_dummy_responses_info.shape[0]

In [22]:
########### Model_3 reflexivity trials dummy comparison

In [23]:
reflexivity_dummy_trials_corpus_df=reflexivity_dummy_info[trials_info_columns]

In [24]:
# encode corpus in positional tokens
encoded_reflexivity_dummy_data = np.array([[token_to_index[token] for token in sequence] for sequence in np.array(reflexivity_dummy_trials_corpus_df)])
reflexivity_dummy_tensor_encoded=torch.from_numpy(encoded_reflexivity_dummy_data).int().to(device)
reflexivity_dummy_tensor_encoded=reflexivity_dummy_tensor_encoded.to(torch.long)

In [25]:
model_reflexivity_dummy = GPTLanguageModel()
m_reflexivity_dummy = model_reflexivity_dummy.to(device)
# print the number of parameters in the model
print(sum(p.numel() for p in m_reflexivity_dummy.parameters())/1e6, 'M parameters')

data=reflexivity_dummy_tensor_encoded
model=m_reflexivity_dummy

# create a PyTorch optimizer
optimizer = torch.optim.AdamW(model_reflexivity_dummy.parameters(), lr=learning_rate)

for iter in range(max_iters):

    # every once in a while evaluate the loss on train and val sets
    if iter % eval_interval == 0 or iter == max_iters - 1:
        losses = estimate_loss()
        # print(losses)
        print(f"step {iter}: train loss {losses['train']:.4f}")

    # sample a batch of data
    xb, yb = get_batch(data)

    # evaluate the loss
    logits, loss = m_reflexivity_dummy(xb, yb)
    optimizer.zero_grad(set_to_none=True)
    loss.backward()
    optimizer.step()

10.682164 M parameters
step 0: train loss 3.9744
step 500: train loss 0.0011
step 1000: train loss 0.0006
step 1500: train loss 0.0003
step 2000: train loss 0.0002
step 2500: train loss 0.0001
step 3000: train loss 0.0001
step 3500: train loss 0.0002
step 4000: train loss 0.0001
step 4500: train loss 0.0001
step 4999: train loss 0.0001


In [26]:
# ### Predict model data
# reflexivity_dummy_responses=get_trials_answers(reflexivity_dummy_tensor_encoded, m_reflexivity_dummy)

# reflexivity_dummy_responses_info=reflexivity_dummy_info.copy()
# reflexivity_dummy_responses_info["response"]=reflexivity_dummy_responses
# reflexivity_dummy_responses_info["response_val"]=(reflexivity_dummy_responses_info.st_comparison==reflexivity_dummy_responses_info.response)*1

# sum(reflexivity_dummy_responses_info.response_val)/reflexivity_dummy_responses_info.shape[0]

In [27]:
########### Model_4 reflexivity trials class members comparison

In [28]:
reflexivity_trials_corpus_df=reflexivity_info[trials_info_columns]

In [29]:
# encode corpus in positional tokens
encoded_reflexivity_data = np.array([[token_to_index[token] for token in sequence] for sequence in np.array(reflexivity_trials_corpus_df)])
reflexivity_tensor_encoded=torch.from_numpy(encoded_reflexivity_data).int().to(device)
reflexivity_tensor_encoded=reflexivity_tensor_encoded.to(torch.long)

In [30]:
model_reflexivity = GPTLanguageModel()
m_reflexivity = model_reflexivity.to(device)
# print the number of parameters in the model
print(sum(p.numel() for p in m_reflexivity.parameters())/1e6, 'M parameters')

data=reflexivity_tensor_encoded
model=m_reflexivity

# create a PyTorch optimizer
optimizer = torch.optim.AdamW(model_reflexivity.parameters(), lr=learning_rate)

for iter in range(max_iters):

    # every once in a while evaluate the loss on train and val sets
    if iter % eval_interval == 0 or iter == max_iters - 1:
        losses = estimate_loss()
        # print(losses)
        print(f"step {iter}: train loss {losses['train']:.4f}")

    # sample a batch of data
    xb, yb = get_batch(data)

    # evaluate the loss
    logits, loss = m_reflexivity(xb, yb)
    optimizer.zero_grad(set_to_none=True)
    loss.backward()
    optimizer.step()

10.682164 M parameters
step 0: train loss 4.0132
step 500: train loss 0.0011
step 1000: train loss 0.0002
step 1500: train loss 0.0005
step 2000: train loss 0.0003
step 2500: train loss 0.0000
step 3000: train loss 0.0000
step 3500: train loss 0.0001
step 4000: train loss 0.0000
step 4500: train loss 0.0000
step 4999: train loss 0.0000


In [31]:
### Predict model data

# reflexivity_responses=get_trials_answers(reflexivity_tensor_encoded, m_reflexivity)

# reflexivity_responses_info=reflexivity_info.copy()
# reflexivity_responses_info["response"]=reflexivity_responses
# reflexivity_responses_info["response_val"]=(reflexivity_responses_info.st_comparison==reflexivity_responses_info.response)*1

# sum(reflexivity_responses_info.response_val)/reflexivity_responses_info.shape[0]

In [32]:
#### Persist_ models
# models_dict={
#     'm_class':m_class,
#     'm_dummy':m_dummy,
#     'm_reflexivity':m_reflexivity,
#     'm_reflexivity_dummy':m_reflexivity_dummy
# }

# with open('salidas/SE_NAnoGPT_4models_option.pickle', 'wb') as f:
#     pickle.dump(models_dict, f, pickle.HIGHEST_PROTOCOL)

In [33]:
#### Load trainded models

# with open('salidas/SE_NAnoGPT_4models.pickle', 'rb') as f:
#     models_dict = pickle.load(f)

# m_class = models_dict['m_class'].to(device)
# m_dummy = models_dict['m_dummy'].to(device)
# m_reflexivity = models_dict['m_reflexivity'].to(device)
# m_reflexivity_dummy = models_dict['m_reflexivity_dummy'].to(device)

In [34]:
########## ALL trials 4 models responses

In [35]:
train_trials=train_info.copy()
train_dummy_trials=train_dummy_info.copy()
reflexivity_trials=reflexivity_info.copy()
reflexivity_dummy_trials=reflexivity_dummy_info.copy()
symmetry_trials=symmetry_info.copy()
transitivity_trials=transitivity_info.copy()

In [36]:
train_dummy_trials.sample_subset="train_dummy"
reflexivity_dummy_trials.sample_subset="reflexivity_dummy"

In [37]:
full_trials=pd.concat([
    train_trials,
    train_dummy_trials,
    reflexivity_trials,
    reflexivity_dummy_trials,
    symmetry_trials,
    transitivity_trials
],ignore_index=True)

In [38]:
trials_corpus_df=full_trials[trials_info_columns]

In [39]:
# encode corpus in positional tokens
encoded_trials_data = np.array([[token_to_index[token] for token in sequence] for sequence in np.array(trials_corpus_df)])
trials_tensor_encoded=torch.from_numpy(encoded_trials_data).int().to(device)
trials_tensor_encoded=trials_tensor_encoded.to(torch.long)

In [40]:
n_trials_report=int(trials_tensor_encoded.shape[0]/10)

In [41]:
print("model 1")
trials_responses_m_class=get_trials_answers(trials_tensor_encoded, m_class, tell_me_n_trials=n_trials_report)
print("model 2")
trials_responses_m_dummy=get_trials_answers(trials_tensor_encoded, m_dummy, tell_me_n_trials=n_trials_report)
print("model 3")
trials_responses_m_reflexivity=get_trials_answers(trials_tensor_encoded, m_reflexivity, tell_me_n_trials=n_trials_report)
print("model 4")
trials_responses_m_reflexivity_dummy=get_trials_answers(trials_tensor_encoded, m_reflexivity_dummy, tell_me_n_trials=n_trials_report)

model 1
0 (0.00%) trials processed
31248 (10.00%) trials processed
62496 (20.00%) trials processed
93744 (30.00%) trials processed
124992 (40.00%) trials processed
156240 (50.00%) trials processed
187488 (60.00%) trials processed
218736 (70.00%) trials processed
249984 (80.00%) trials processed
281232 (90.00%) trials processed
312479 (100.00%) trials processed
model 2
0 (0.00%) trials processed
31248 (10.00%) trials processed
62496 (20.00%) trials processed
93744 (30.00%) trials processed
124992 (40.00%) trials processed
156240 (50.00%) trials processed
187488 (60.00%) trials processed
218736 (70.00%) trials processed
249984 (80.00%) trials processed
281232 (90.00%) trials processed
312479 (100.00%) trials processed
model 3
0 (0.00%) trials processed
31248 (10.00%) trials processed
62496 (20.00%) trials processed
93744 (30.00%) trials processed
124992 (40.00%) trials processed
156240 (50.00%) trials processed
187488 (60.00%) trials processed
218736 (70.00%) trials processed
249984 (80.

In [42]:
full_trials["response_m_class"]=trials_responses_m_class
full_trials["response_m_dummy"]=trials_responses_m_dummy
full_trials["response_m_reflexivity"]=trials_responses_m_reflexivity
full_trials["response_m_reflexivity_dummy"]=trials_responses_m_reflexivity_dummy

In [43]:
full_trials

,sample_subset,st_sample,st_comp1,st_comp2,st_comp3,option_answer,st_comparison,response_m_class,response_m_dummy,response_m_reflexivity,response_m_reflexivity_dummy
0,train,A1,B1,E4,C4,O_1,B1,O_1,O_3,O_1,O_1
1,train,A1,E4,B1,C4,O_2,B1,O_2,O_3,O_2,O_3
2,train,A1,E4,C4,B1,O_3,B1,O_3,O_3,O_3,O_3
3,train,A1,B1,E4,D4,O_1,B1,O_1,O_1,O_1,O_3
4,train,A1,E4,B1,D4,O_2,B1,O_2,O_3,O_2,O_3
...,...,...,...,...,...,...,...,...,...,...,...
312475,transitivity,G4,D1,F4,B2,O_2,F4,O_1,O_3,O_2,O_3
312476,transitivity,G4,D1,B2,F4,O_3,F4,O_3,O_3,O_3,O_3
312477,transitivity,G4,F4,D1,C1,O_1,F4,O_1,O_1,O_1,O_1
312478,transitivity,G4,D1,F4,C1,O_2,F4,O_2,O_3,O_2,O_3


In [44]:
# Export Output

In [45]:
with open('salidas/encoder_trials_responses_option_OTM.pickle', 'wb') as f:
    pickle.dump(full_trials, f, pickle.HIGHEST_PROTOCOL)